In [1]:
import numpy as _numpy_
from ray.tune.schedulers import PopulationBasedTraining
from ray import tune , air
from controllables.energyplus import System
from controllables.core import TemporaryUnavailableError
from controllables.core.tools.gymnasium import DiscreteSpace, BoxSpace, DictSpace
from controllables.core.tools.rllib import MultiAgentEnv
from controllables.energyplus import Actuator, OutputVariable, OutputMeter
from controllables.energyplus import examples
import pythermalcomfort as pytc
import pandas as pd



class RewardFunction:
    def __init__(self, metab_rate=1.5, clothing=.5, pmv_limit=.5):
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
        self.df_baseline = pd.read_csv("datasave/records_-1FWEST.csv")
        self.df_baseline['time'] = pd.to_datetime(self.df_baseline['time'])
    
   
    def __call__(self, agent):
        try:
            system = agent.system
            observation = agent.observation.value    
        except TemporaryUnavailableError:
            return 0. 
        AHU_COOLING_COIL = observation['AHU COOLING COIL']
        HVAC_Electricity = observation['energy-consumption']
        # Fan_Electricity_Rate = observation['Fan Electricity Rate']
        Office_Occupancy = observation['Office Occupancy']
        pmv = pytc.models.pmv_ppd(
            tdb=(tdb := observation['temperature:drybulb']), 
            tr=observation['temperature:radiant'], 
            # calculate relative air speed
            vr=pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
            rh=observation['humidity'], 
            met=self._metab_rate, 
            # calculate dynamic clothing
            clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
            limit_inputs=False ,
        )['pmv']
        current_time = system['time'].value
        penalty = 0
        if Office_Occupancy != 0:  
            if HVAC_Electricity > self.df_baseline[self.df_baseline['time'] == current_time]['elec'].max():
                penalty += 10
            # if _numpy_.abs(pmv) > _numpy_.abs(self.df_baseline[self.df_baseline['time'] == current_time]['pmv']).max():
            #     penalty += 10

        log_term = max(2 * _numpy_.abs(pmv), 1e-2) 
        
        reward = (
            2 * Office_Occupancy * (-_numpy_.log(log_term))  
            - penalty
        )
        reward = min(reward, 10)
        return reward


class UserMultiAgentEnv(MultiAgentEnv):
    config: MultiAgentEnv.Config = {
        'agents': {
            # room agents
            **{
                agent_id: {
                    'action_space': DictSpace({
                        'thermostat': BoxSpace(
                            low=20., high=30.,
                            dtype=_numpy_.float32,
                            shape=(),
                        ).bind(
                            Actuator.Ref(
                                type='Zone Temperature Control',
                                control_type='Cooling Setpoint',
                                key=var_key,
                            ) 
                        ),
                        # 'air-flow-rate': BoxSpace(
                        #     low=0., high=20.,
                        #     dtype=_numpy_.float32,
                        #     shape=(),
                        # ).bind(
                        #     Actuator.Ref(
                        #         type='System Node Setpoint',
                        #         control_type='Mass Flow Rate Setpoint',
                        #         key=f'{var_key} SINGLE DUCT VAV NO REHEAT SUPPLY OUTLET',
                        #     )
                        # ),                       
                    }),
                    'observation_space': DictSpace({
                        # 'temperature': BoxSpace(
                        #     low=-_numpy_.inf, high=+_numpy_.inf,
                        #     dtype=_numpy_.float32,
                        #     shape=(),
                        # ).bind(
                        #     OutputVariable.Ref(
                        #         type='Zone Air Temperature',
                        #         key=var_key,
                        #     )
                        # ),
                        'temperature:drybulb': BoxSpace(
                                low=-_numpy_.inf, high=+_numpy_.inf,
                                dtype=_numpy_.float32,
                                shape=(),
                            ).bind(OutputVariable.Ref(
                                type='Zone Mean Air Temperature',
                                key=var_key,
                            )),
                        'temperature:radiant': BoxSpace(
                                low=-_numpy_.inf, high=+_numpy_.inf,
                                dtype=_numpy_.float32,
                                shape=(),
                            ).bind(OutputVariable.Ref(
                                type='Zone Mean Radiant Temperature',
                                key=var_key,
                            )),
                        'humidity': BoxSpace(
                                low=-_numpy_.inf, high=+_numpy_.inf,
                                dtype=_numpy_.float32,
                                shape=(),
                            ).bind(OutputVariable.Ref(
                                type='Zone Air Relative Humidity',
                                key=var_key,
                            )),
                        'AHU COOLING COIL': BoxSpace(
                                low=-_numpy_.inf, high=+_numpy_.inf,
                                dtype=_numpy_.float32,
                                shape=(),
                            ).bind(OutputVariable.Ref(
                                type='Cooling Coil Total Cooling Rate',
                                key='AIR LOOP AHU COOLING COIL',
                            )),
                        'Fan Electricity Rate': BoxSpace(
                            low=-_numpy_.inf, high=+_numpy_.inf,
                            dtype=_numpy_.float32,
                            shape=(),
                        ).bind(OutputVariable.Ref(
                            type='Fan Electricity Rate',
                            key='AIR LOOP AHU SUPPLY FAN',
                        )),
                        'energy-consumption': BoxSpace(
                            low=-_numpy_.inf, high=+_numpy_.inf,
                            dtype=_numpy_.float32,
                            shape=(),
                        ).bind(OutputMeter.Ref(
                            type='Electricity:HVAC',
                        )
                        ),
                        'Office Occupancy':BoxSpace(
                            low=-_numpy_.inf, high=+_numpy_.inf,
                            dtype=_numpy_.float32,
                            shape=(),
                        ).bind(OutputVariable.Ref(
                            type='Schedule Value',
                            key='Office_OpenOff_Occ',
                        )),
                    }),
                    'reward': RewardFunction(),
                }
                for agent_id, var_key in [
                    ('1FWEST', '1FFIRSTFLOORWEST:OPENOFFICE'),
                    ('1FEAST', '1FFIRSTFLOOREAST:OPENOFFICE'),
                    ('0FWEST', '0FGROUNDFLOORWEST:OPENOFFICE'),
                    ('0FEAST', '0FGROUNDFLOOREAST:OPENOFFICE'),
                    ('1FWEST1', '1FFIRSTFLOORWEST1:OPENOFFICE'),
                    ('1FEAST1', '1FFIRSTFLOOREAST1:OPENOFFICE'),
                    ('0FWEST1', '0FGROUNDFLOORWEST1:OPENOFFICE'),
                    ('0FEAST1', '0FGROUNDFLOOREAST1:OPENOFFICE'),
                ]
            },
            # HVAC equipment agents
            'CHILLER': {
                'participation': lambda agent: False,
                'action_space': DictSpace({
                    'temperature': BoxSpace(
                        low=-_numpy_.inf, high=+_numpy_.inf,
                        dtype=_numpy_.float32,
                        shape=(),
                    ).bind(Actuator.Ref(
                        'System Node Setpoint',
                        'Temperature Setpoint',
                        'CHILLER CHW OUTLET NODE',
                    )),
                }),
                'observation_space': DictSpace({
                    'cooling-rate': BoxSpace(
                        low=-_numpy_.inf, high=+_numpy_.inf,
                        dtype=_numpy_.float32,
                        shape=(),
                    ).bind(OutputVariable.Ref(
                        'Chiller Evaporator Cooling Rate',
                        'CHILLER',
                    )),
                    'elec-rate': BoxSpace(
                        low=-_numpy_.inf, high=+_numpy_.inf,
                        dtype=_numpy_.float32,
                        shape=(),
                    ).bind(OutputVariable.Ref(
                        'Chiller Electricity Rate',
                        'CHILLER',
                    )),
                }),
                'reward': (
                    lambda agent: (
                        agent.observation.value['cooling-rate'] 
                        / agent.observation.value['elec-rate']
                    ) if agent.observation.value['elec-rate'] != 0. else 0.
                ),
            },
            'AHU': {
                # TODO disabled
                'participation': lambda agent: False,
                'action_space': DictSpace({
                    'air flow rate': BoxSpace(
                        low=0., high=10.,
                        dtype=_numpy_.float32,
                        shape=(),
                    ).bind(Actuator.Ref(
                        type='Fan',
                        control_type='Fan Air Mass Flow Rate',
                        key='AIR LOOP AHU SUPPLY FAN'
                    )),
                }),
                'observation_space': DictSpace({
                    'energy-consumption': BoxSpace(
                        low=-_numpy_.inf, high=+_numpy_.inf,
                        dtype=_numpy_.float32,
                        shape=(),
                    ).bind(
                        OutputMeter.Ref(
                            type='Electricity:HVAC',
                        )
                    ),
                    'AHU COOLING COIL': BoxSpace(
                        low=-_numpy_.inf, high=+_numpy_.inf,
                        dtype=_numpy_.float32,
                        shape=(),
                    ).bind(OutputVariable.Ref(
                        type='Cooling Coil Total Cooling Rate',
                        key='AIR LOOP AHU COOLING COIL',
                    )),
                }),
                'reward': (
                    lambda agent: (
                        agent.observation.value['AHU COOLING COIL'] 
                        / agent.observation.value['energy-consumption']
                    ) if agent.observation.value['energy-consumption'] != 0. else 0.
                ),
            },            
        }
    }

    def __init__(self, config: dict = dict()):
        super().__init__({
            **self.__class__.config,
            **config,
        })

    def run(self):
        system = System(
            building='all_room_have_hvac.idf',
            weather='SGP_Singapore_486980_IWEC.epw',
            # TODO
            report='tmp/',
            repeat=True,
        )
        # system.add('logging:progress')
        self.__attach__(system).schedule_episode()
        system.start().wait()

E0000 00:00:1731316989.611473 1149134 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731316989.614829 1149134 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.algorithms.ppo import PPO, PPOConfig

def get_config():
    return (
        PPOConfig()
        .environment(UserMultiAgentEnv)
        .env_runners(
            # NOTE this env (an `ExternalEnv`) does not support connectors
            enable_connectors=False,
        )
        .multi_agent(
            policies={
                policy_id: PolicySpec(
                    action_space=agent_config['action_space'],
                    observation_space=agent_config['observation_space'],
                )
                for policy_id, agent_config in UserMultiAgentEnv.config['agents'].items()
            },
            policy_mapping_fn=lambda agent_id, *args, **kwargs: str(agent_id),
        )
        # TODO
        .resources(num_gpus=1)
    )

In [3]:
pbt = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=4,
    resample_probability=0.25,
    hyperparam_mutations={
        "lr": tune.uniform(1e-5, 0.1),
        "batch_mode": ["complete_episodes"],
        "train_batch_size": [4000],
        "sgd_minibatch_size": [32, 64, 128, 256, 512],
        "num_sgd_iter": [10, 20, 30],
        "clip_param": tune.uniform(0.1, 0.3),
    },
    #require_attrs=False,
)

tuner = tune.Tuner(
    "PPO",
    param_space=get_config().to_dict(),
    tune_config=tune.TuneConfig(
        scheduler=pbt,
        num_samples=1,
        metric="env_runners/episode_reward_mean",
        mode="max",
    ),
    run_config=air.RunConfig(
        stop={"training_iteration": 200},
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True
        ),
        verbose=2,
    )
)

results = tuner.fit()


/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1362: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-11-11_09-23-15_544064_1149134/logs/gcs_server.out' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1362: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-11-11_09-23-15_544064_1149134/logs/gcs_server.err' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1367: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-11-11_09-23-15_544064_1149134/logs/monitor.out' mode='a' encoding='utf-8'>
  self.start_monitor()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1367: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session

(pid=1150781) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=1150781) E0000 00:00:1731316997.606674 1150781 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=1150781) E0000 00:00:1731316997.610106 1150781 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=1150898) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=1150898) E0000 00:00:1731317004.682765 1150898 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=1150898) E0000 00:00:1731317004.686129 1150898 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=1150899) WARNING: All log messa

Trial name,agent_timesteps_total,counters,custom_metrics,episode_media,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,timers
PPO_UserMultiAgentEnv_95577_00000,6400000,"{'num_env_steps_sampled': 800000, 'num_env_steps_trained': 800000, 'num_agent_steps_sampled': 6400000, 'num_agent_steps_trained': 6400000}",{},{},6400000,6400000,6400000,800000,800000,4000,68.2054,800000,4000,68.2054,2,0,0,4000,"{'cpu_util_percent': 7.025, 'ram_util_percent': 60.83333333333332}","{'training_iteration_time_ms': 59350.0, 'restore_workers_time_ms': 0.012, 'training_step_time_ms': 59349.971, 'sample_time_ms': 28158.075, 'learn_time_ms': 31185.322, 'learn_throughput': 128.265, 'synch_weights_time_ms': 6.278}"


(PPO pid=1150781) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/AD/user/ray_results/PPO_2024-11-11_09-23-15/PPO_UserMultiAgentEnv_95577_00000_0_2024-11-11_09-23-16/checkpoint_000000)
2024-11-11 13:02:05,801	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/AD/user/ray_results/PPO_2024-11-11_09-23-15' in 0.0911s.
2024-11-11 13:02:06,114	INFO tune.py:1041 -- Total run time: 13129.40 seconds (13128.97 seconds for the tuning loop).


In [4]:
best_result = results.get_best_result()
print("Best Hyperparameters found: ", best_result)

Best Hyperparameters found:  Result(
  metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'1FWEST': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 15.585890248945605, 'cur_kl_coeff': 9.262305278225771e-20, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 8.133694656193256, 'policy_loss': -0.0004515602010845517, 'vf_loss': 8.134146212538083, 'vf_explained_var': -4.121921956539154e-05, 'kl': 0.010957721841077964, 'entropy': -1.7783307994405428, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0, 'num_grad_updates_lifetime': 191520.5, 'diff_num_grad_updates_vs_sampler_policy': 479.5}, '0FWEST': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 32.88093289838483, 'cur_kl_coeff': 3.469446951953615e-19, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 8.731304128964743, 'policy_loss': 0.003165417871787213, 'vf_loss': 8.72813870559136, 'vf_explained_var': -6.829698880513509e-10, 'kl': 0.008857792762683244, 'entro

In [6]:
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from controllables.core import BaseVariable
from controllables.core.tools.records import VariableRecords

class PMVVariable(BaseVariable):
    def __init__(
        self, 
        tdb: BaseVariable,
        tr: BaseVariable,
        rh: BaseVariable,
        metab_rate=1.5, clothing=.5, pmv_limit=.5,
    ):
        self.tdb = tdb
        self.tr = tr
        self.rh = rh
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
    
    @property
    def value(self):
        res = pytc.models.pmv_ppd(
            tdb=self.tdb.value, 
            tr=self.tr.value, 
            # calculate relative air speed
            vr=pytc.utilities.v_relative(v=0.1, met=self._metab_rate), 
            rh=self.rh.value, 
            met=self._metab_rate, 
            # calculate dynamic clothing
            clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
            limit_inputs=False,
        )['pmv']

        return res

class PlottingCallbacks(DefaultCallbacks):
    def __init__(self):
        self.env_records: dict[object, VariableRecords] = None

    def on_episode_start(self, *, episode, worker, **kwargs):
        env: UserMultiAgentEnv = worker.env
        system = env.system
        if self.env_records is None:
            system.add('logging:progress')
            self.env_records = dict()
            for agent_ref in env.agents:
                if agent_ref == 'CHILLER' or agent_ref == 'AHU':
                    # self.env_records[agent_ref] = records = VariableRecords({
                    # 'AHU COOLING COIL': system[OutputVariable.Ref('Cooling Coil Total Cooling Rate', 'AIR LOOP AHU COOLING COIL')],
                    # 'Electricity': system[OutputMeter.Ref('Electricity:HVAC')]
                    # }, maxlen=10_000)
                    pass
                else:
                    tdb = env.agents[agent_ref].observation['temperature:drybulb']
                    tr = env.agents[agent_ref].observation['temperature:radiant']
                    rh = env.agents[agent_ref].observation['humidity']
                    pmv = PMVVariable(tdb=tdb, tr=tr, rh=rh)
                    self.env_records[agent_ref] = records = VariableRecords({
                        '🕰️': system['time'],
                        '🍩': env.agents[agent_ref].reward,
                        'pmv': pmv,
                        'occupancy': env.agents[agent_ref].observation['Office Occupancy'],
                        # 'tstat': env.agents[agent_ref].action['thermostat'],
                        'temp': tdb,
                        # 'AHU COOLING COIL': system[OutputVariable.Ref('Cooling Coil Total Cooling Rate', 'AIR LOOP AHU COOLING COIL')],
                        'elec': system[OutputMeter.Ref('Electricity:HVAC')],
                    }, maxlen=10_000)
                    # display(
                    #     records.plot.scatter(x='🕰️', y='pv', label=repr(agent_ref))
                    #     .watch(records.events['change'] % 1_000)
                    # )

    def on_episode_step(self, *, episode, **kwargs):
        for _, env_records in self.env_records.items():
            env_records.poll()

    def on_episode_end(self, *, episode, **kwargs):
        for agent_ref, env_records in self.env_records.items():
            env_records.dataframe().to_csv(f'datasave/20241110/records_train_tradiction-{agent_ref}.csv', index=False)


In [7]:
config_eval = (
    get_config()
    .env_runners(
        num_env_runners=0,
        create_env_on_local_worker=True,
    )
    .evaluation(
        evaluation_duration=1,
        evaluation_duration_unit='episodes',
        #evaluation_interval=1,
        evaluation_num_env_runners=0,
    )
    .callbacks(PlottingCallbacks)
)

algo_eval = PPO(config_eval)
#algo_eval.restore(best_result.checkpoint)
algo_eval.restore('/home/AD/user/ray_results/PPO_2024-11-10_04-06-03/PPO_UserMultiAgentEnv_1bb96_00001_1_2024-11-10_04-06-06/checkpoint_000004')
#algo_eval.restore("/home/AD/user/ray_results/PPO_2024-10-18_15-32-34/PPO_UserMultiAgentEnv_331fb_00000_0_2024-10-18_15-32-35/checkpoint_000013")

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:555: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationW

In [8]:
for _ in range(2):
    algo_eval.evaluate()

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:345: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Zone Mean Air Temperature', key='1FFIRSTFLOORWEST:OPENOFFICE')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
  _warnings_.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:345: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Fan Electricity Rate', key='AIR LOOP AHU SUPPLY FAN')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
  _warnings_.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:345

  0%|          | 0/100 [00:00<?, ?it/s]